In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import mlflow
from pprint import pprint
import pickle

In [4]:
def fetch_logged_data(run_id):
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

# enable autologging
mlflow.sklearn.autolog()

2021/11/14 20:55:12 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


In [5]:
house = pd.read_csv("../data/clean_houses.csv")
house.head()
feature = house.drop(columns=["Price"])
price = house["Price"]

In [6]:
feature.head()
price.head()

0     28000000.0
1     24000000.0
2     35000000.0
3    112000000.0
4     12000000.0
Name: Price, dtype: float64

In [7]:
feature = feature.to_numpy()
price = price.to_numpy()

In [8]:
np.isnan(feature.any()) #and gets False
# np.isfinite(feature.all()) #and gets True
np.max(price)

2200000000.0

In [9]:
X_train, X_test, y_train, y_test = train_test_split(feature, price, test_size=0.33, random_state=42)

In [23]:
model = LinearRegression()
with mlflow.start_run() as run:
    reg = model.fit(X_train, y_train)
reg.score(X_train, y_train)

pickle.dump(model, open("../data/model.sav", "wb"))

In [25]:
print(feature[0])
reg.predict(feature[0].reshape(1,-1))

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 3 4 4]


array([66759987.34646924])

In [13]:
price[0]

28000000.0

In [14]:
# fetch logged data
params, metrics, tags, artifacts = fetch_logged_data(run.info.run_id)

pprint(params)
pprint(metrics)
pprint(tags)
pprint(artifacts)

{'copy_X': 'True',
 'fit_intercept': 'True',
 'n_jobs': 'None',
 'normalize': 'deprecated',
 'positive': 'False'}
{'LinearRegression_score_X_train': 0.045566148857284605,
 'training_mae': 93567082.69944245,
 'training_mse': 6.228168003160279e+16,
 'training_r2_score': 0.045566148857284605,
 'training_rmse': 249562978.08690053,
 'training_score': 0.045566148857284605}
{'estimator_class': 'sklearn.linear_model._base.LinearRegression',
 'estimator_name': 'LinearRegression'}
['model/MLmodel',
 'model/conda.yaml',
 'model/model.pkl',
 'model/requirements.txt']
